In [ ]:
import pormake as pm
import numpy as np
import csv

db = pm.Database()
loc = pm.Locator()

## metal nodes
Fe_oct_a = pm.BuildingBlock(bb_file="pi-d_building_blocks/old_bb/Fe_oct.xyz")
Fe_oct_b = Fe_oct_a.make_chiral_building_block()
Fe_oct = Fe_oct_a

## organic nodes
TP_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TP_O.xyz")

## edges
spacer = pm.BuildingBlock(bb_file="pi-d_building_blocks/spacer.xyz")
DHBQ = pm.BuildingBlock(bb_file="pi-d_building_blocks/DHBQ.xyz")
Cu_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Cu_pln.xyz")
pseudo_edge = pm.BuildingBlock(bb_file="pi-d_building_blocks/pseudo_edge.xyz")

pm.log.disable_print()
pm.log.disable_file_print()


## Prepare topologies for screening

In [ ]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [ ]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [ ]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

node_limit = 80
cn33_topos = []

for topo in topos:
    
    if topo.n_nodes > node_limit:
        continue
    
    if max(topo.unique_cn) == 3:
        cn33_topos.append(topo)

print(len(cn33_topos))

## Perform screening (try different node & enantionmer arrangements)

In [ ]:
TP_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TP_O.xyz")

## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=30, check_tetrahedral=False)
successful_cases = []

## control group size for alternation

ML_edge = spacer
MM_edge = DHBQ
LL_edge = Cu_pln

for topo in cn33_topos:
    
    num_nodes = len(topo.node_indices)
    
    for type_grouping in [1, 2, 3, 4, 6, 8]:

        ## avoid having only one type of node
        if type_grouping >= num_nodes:
            continue
        
        for type_begin_flip in [True, False]:           
            
            for chiral_grouping in [1, 2, 3, 4, 6, 8, 12, 16, 81]:
                
                for chiral_begin_flip in [True, False]:       

                    current_node = {}
                    current_edge = {}    
                    
                    ## initialize nodes with TP_O first (no chirality dependence... is this good?)
                    for i, node in enumerate(topo.unique_cn):
                        current_node[i] = TP_O
                
                    ## initialize edges
                    for i, edge in enumerate(topo.unique_edge_types):
                        current_edge[tuple(edge)] = spacer
                
                    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
                    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
                    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
                    
                    ## treat cn=4 cases. we do not have metal-based exp. cn=4 bbs, so enforce PcM linker for all cn=4 nodes.
                    ## NOTE: For now, we leave out cases where cn=3 linkers are also mixed in.
                
                    type_counter = 0
                    chiral_counter = 0

                    type_flip = type_begin_flip
                    chiral_flip = chiral_begin_flip
                   
                    for node in topo.node_indices:
                        
                        type_counter += 1                

                        if type_flip:
                            bbs[node] = TP_O
                        else:
                            bbs[node] = Fe_oct
                        
                        if type_counter % type_grouping == 0:
                            type_flip = not type_flip
                        
                        ## chirality
                        if bbs[node].name == "Fe_oct":
                            
                            chiral_counter += 1                
                            
                            if chiral_flip:
                                bbs[node] = Fe_oct_b
                            else:
                                bbs[node] = Fe_oct_a
                            
                            if chiral_counter % chiral_grouping == 0:
                                chiral_flip = not chiral_flip
                    
                    ## iterate through edges and assign edges accordingly
                    for edge in topo.edge_indices:
            
                        node_a, node_b = topo.get_neighbor_indices(edge)
            
                        ## metal-metal case
                        if bbs[node_a].name == 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
                            bbs[edge] = MM_edge
            
                        ## metal-linker case
                        elif bbs[node_a].name == 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
            
                            bbs[edge] = ML_edge
                            ## take care of spacer 
                            calc_permutation[edge] = [1, 0]
            
                        ## linker-metal case
                        elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
            
                            bbs[edge] = ML_edge
                            ## take care of spacer             
                            calc_permutation[edge] = [0, 1]
            
                        ## linker-linker case
                        elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
                            bbs[edge] = LL_edge
            
                        else:
                            print('something\'s wrong!')
            
            
                    try:
                        mof = test_builder.build(topo, bbs, calc_permutation)
                        mof.view()
                        successful_cases.append((topo.name, type_grouping, type_begin_flip, chiral_grouping, chiral_begin_flip))
                        print(topo.name, type_grouping, type_begin_flip, chiral_grouping, chiral_begin_flip)
            
                    except:
                        continue
            

## Raw screening results

In [ ]:
successful_cases = [('bcu-h', 1, True, 4, True),
 ('bpb', 6, True, 3, True), #
 ('bpc', 8, True, 2, True), #
 ('bpc', 8, True, 2, False), 
 ('bpg', 8, True, 2, True), 
 ('bpg', 8, True, 2, False), #
 ('bto', 2, True, 1, True), 
 ('bto', 2, True, 1, False), #
 ('clh', 2, False, 2, True), # 
 ('etb', 6, False, 3, False), #
 ('etd', 6, True, 6, True), #
 ('etd', 6, True, 8, True),
 ('etd', 6, True, 12, True),
 ('etd', 6, True, 16, True),
 ('etd', 6, True, 81, True),
 ('etd', 6, False, 6, True),
 ('etd', 6, False, 8, True),
 ('etd', 6, False, 12, True),
 ('etd', 6, False, 16, True),
 ('etd', 6, False, 81, True),
 ('ete', 6, False, 6, True), #
 ('ete', 6, False, 8, True),
 ('ete', 6, False, 12, True),
 ('ete', 6, False, 16, True),
 ('ete', 6, False, 81, True),
 ('etg', 8, True, 4, False), # 
 ('eth', 8, False, 4, True), #
 ('etj', 8, False, 4, False), #
 ('lig', 1, True, 2, False), #
 ('lig', 1, False, 2, False),
 ('nod', 8, True, 2, False), #
 ('noj', 4, True, 2, True),
 ('noj', 4, True, 2, False),
 ('noj', 4, False, 4, True),
 ('noj', 4, False, 6, True),
 ('noj', 4, False, 8, True),
 ('noj', 4, False, 12, True),
 ('noj', 4, False, 16, True),
 ('noj', 4, False, 81, True), #
 ('pbz', 2, True, 6, True),
 ('srs', 1, True, 4, False),
 ('srs', 1, True, 6, False),
 ('srs', 1, True, 8, False),
 ('srs', 1, True, 12, False),
 ('srs', 1, True, 16, False),
 ('srs', 1, True, 81, False),
 ('srs', 1, False, 4, False),
 ('srs', 1, False, 6, False),
 ('srs', 1, False, 8, False),
 ('srs', 1, False, 12, False),
 ('srs', 1, False, 16, False),
 ('srs', 1, False, 81, False), #
 ('sxc-d', 6, True, 6, False),
 ('sxc-d', 6, True, 8, False),
 ('sxc-d', 6, True, 12, False),
 ('sxc-d', 6, True, 16, False),
 ('sxc-d', 6, True, 81, False), # ??
 ('ths', 1, True, 2, True),
 ('ths', 1, True, 2, False),
 ('ths', 1, False, 2, True),
 ('ths', 1, False, 2, False), #
 ('twt', 2, True, 1, False),
 ('twt', 4, True, 1, True),
 ('twt', 4, True, 1, False),
 ('uni-d', 6, True, 6, False),
 ('uni-d', 6, True, 8, False),
 ('uni-d', 6, True, 12, False),
 ('uni-d', 6, True, 16, False),
 ('uni-d', 6, True, 81, False),
 ('utp', 6, True, 2, True),
 ('utp', 6, True, 3, True),
 ('utp', 6, True, 4, True),
 ('utp', 6, True, 6, True),
 ('utp', 6, True, 8, True),
 ('utp', 6, True, 12, True),
 ('utp', 6, True, 16, True),
 ('utp', 6, True, 81, True)]

## Inspection of raw screening results

Structures from screening are visualized to qualitatively assess their synthesizability. Here, the angles between two organic linker nodes connected together were considered with extra care. In instances where multiple successful cases are reported for the same topology, case with the smallest residual angle and/or highest symmetry is kept.

In [ ]:

pm.log.enable_print()
pm.log.enable_file_print()

## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=30, check_tetrahedral=False)

## control group size for alternation

ML_edge = spacer
MM_edge = DHBQ
LL_edge = Cu_pln

for case in successful_cases:
    
    topo = db.get_topo(case[0])
    type_grouping = case[1]
    type_begin_flip = case[2]    
    chiral_grouping = case[3]
    chiral_begin_flip = case[4]
    
    current_node = {}
    current_edge = {}    
    
    ## initialize nodes with TP_O first (no chirality dependence... is this good?)
    for i, node in enumerate(topo.unique_cn):
        current_node[i] = TP_O

    ## initialize edges
    for i, edge in enumerate(topo.unique_edge_types):
        current_edge[tuple(edge)] = spacer

    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
    ## treat cn=4 cases. we do not have metal-based exp. cn=4 bbs, so enforce PcM linker for all cn=4 nodes.
    ## NOTE: For now, we leave out cases where cn=3 linkers are also mixed in.

    type_counter = 0
    chiral_counter = 0

    type_flip = type_begin_flip
    chiral_flip = chiral_begin_flip
   
    for node in topo.node_indices:
        
        type_counter += 1                

        if type_flip:
            bbs[node] = TP_O
        else:
            bbs[node] = Fe_oct
        
        if type_counter % type_grouping == 0:
            type_flip = not type_flip
        
        ## chirality
        if bbs[node].name == "Fe_oct":
            
            chiral_counter += 1                
            
            if chiral_flip:
                bbs[node] = Fe_oct_b
            else:
                bbs[node] = Fe_oct_a
            
            if chiral_counter % chiral_grouping == 0:
                chiral_flip = not chiral_flip
    
    ## iterate through edges and assign edges accordingly
    for edge in topo.edge_indices:

        node_a, node_b = topo.get_neighbor_indices(edge)

        ## metal-metal case
        if bbs[node_a].name == 'Fe_oct' and bbs[node_b].name == 'Fe_oct':
            bbs[edge] = MM_edge

        ## metal-linker case
        elif bbs[node_a].name == 'Fe_oct' and bbs[node_b].name != 'Fe_oct':

            bbs[edge] = ML_edge
            ## take care of spacer 
            calc_permutation[edge] = [1, 0]

        ## linker-metal case
        elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name == 'Fe_oct':

            bbs[edge] = ML_edge
            ## take care of spacer             
            calc_permutation[edge] = [0, 1]

        ## linker-linker case
        elif bbs[node_a].name != 'Fe_oct' and bbs[node_b].name != 'Fe_oct':
            bbs[edge] = LL_edge

        else:
            print('something\'s wrong!')


    mof = test_builder.build(topo, bbs, calc_permutation)
    mof.view()
    print(topo.name, type_grouping, type_begin_flip, chiral_grouping, chiral_begin_flip)


## Final topoologies to be kept

In [ ]:
keep_cases = [successful_cases[i] for i in [9, #etb
                                            25, #etg
                                            26, #eth
                                            27, #etj
                                            28, #lig
                                            30, #nod
                                            38, #noj
                                            51, #srs
                                            60, #ths
                                            ]]

In [ ]:
keep_cases = [('etb', 6, False, 3, False),
 ('etg', 8, True, 4, False),
 ('eth', 8, False, 4, True),
 ('etj', 8, False, 4, False),
 ('lig', 1, True, 2, False),
 ('nod', 8, True, 2, False),
 ('noj', 4, False, 81, True),
 ('srs', 1, False, 81, False),
 ('ths', 1, False, 2, False)]